# CH07.Docker docs

### 1. $ docker --help
 당연하게도 다음과 같이 ```docker --help```를 사용하여 docker를 사용하면된다.

In [3]:
!docker --help


Usage:  docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/Users/leibniz/.docker")
  -c, --context string     Name of the context to use to connect to the
                           daemon (overrides DOCKER_HOST env var and
                           default context set with "docker context use")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/Users/leibniz/.docker/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/Users/leibniz/.docker/cert.pem"

또는 위처럼 https://docs.docker.com/go/guides/에 접속하여 공식문서를 참조하는게 당연하고 일반적인 방식이다. 하지만 당연히 docker를 사람들이 사용하며 노하우가 쌓일 것이고, 그러다보면 자주 사용하고, 사용하면 유용한 command가 존재한다. 몇 개만 알아보고 지나가면 좋을 것 같다.

### 2. Some useful command in Docker
#### 2.1. docker run
Image로 container를 만드는 동시에 실행까지 시키는 커맨드이다. 이에 대해서는 유용한 option들이 다음과 같이 존재한다.<br>
(2021.02.16 추가) : mysql 같은 서비스를 가상화할 때, 환경변수를 다루는 옵션이 추가로 필요하다.
- \-p \[HOSTPORT\]:\[GUESTPORT\] : 포트포워딩을 할 수 있는 옵션이다.
- \-d : 백그라운드에서 실행할 수 있는 옵션이다.
- \--rm : 컨테이너 종료시 자동으로 컨테이너가 삭제되는 옵션이다. 임시 컨테이너 생성시 주로 사용한다.
- \--name \[CONTAINER_NAME\] : 컨테이너에 이름을 지정해 줄 수 있다. id처럼 사용가능하여 유용하다.
- \-e \[VAR_NAME\]=\[VAR_VALUE\] : 컨테이너에 환경변수를 추가해 줄 수 있다.

#### 2.2. docker exec -it tc /bin/bash
Container 내부의 shell을 실행하는 command이다. 물론 대부분의 image들이 경량화를 하여 만들어지지만, 이 command를 이용하여 가상환경 내부의 상황을 조금더 쉽게 확인할 수 있다. 

- \-i : Host의 stdin을 container로 input을 줄 수 있다.
- \-t : Container의 output을 host의 stdout으로 받을 수 있다.

굳이 shell을 실행하지 않더라도, ```docker exec -it [SHELL_COMMAND]```를 이용하여 command를 줄 수 있다.

#### 2.3. docker logs \[CONTAINER_NAME\]
Container의 log를 확인할 수 있다. 이 container를 구축할 때, 어떤 log를 남겼는지 확인할 수 있고, Redirection을 이용하여 파일로 만들어 보는 것이 일반적이다.

- ```docker logs [CONTAINER_NAME] > logfile.txt # stdout```
- ```docker logs [CONTAINER_NAME] &> logfile.txt # stderr```

#### 2.4. File transfer between host and container or between containers
당연히 docker를 사용하다보면 Host에서 Container로 또는 Container에서 Container로 파일을 옮겨야하는 경우가 생긴다. 다음과 같은 command를 이용하여 옮길 수 있다.

- ```docker cp [FROM_PATH] [TO_PATH]```
이때, host의 경우는 일반적인 file path를 사용하면 된다. 하지만 container의 경우는 다음과 같이 file path를 구성한다. ```[CONTAINER_NAME]:[FILE_PATH]``` 이렇게하여 host와 container를, container와 container를 구별할 수 있다.

#### 2.5. Removing all container
먼저 ```docker ps```에서 ```-q```옵션을 주어서 container의 id들을 모두 출력받을 수 있다. 이를 이용하여 다음과 같이 한번에 모든 container를 중지/삭제시킬 수 있다.

```docker stop 'docker ps -a -q'```<br>
```docker rm 'docker ps -a -q'```<br>
<br><br>

이렇게 여러가지 옵션과 command, linux 명령어들을 혼합하여 사용하면 굉장히 효율적으로 사용할 수 있다.

### 3. 볼륨 마운트
Docker에서 Container에 데이터를 저장하는 방법은 크게 두 가지가 있다. <br>
첫 번째는 volume을 이용하는 방법이고, 이는 docker에서 권장하는 방식이다. 다음과 같은 커맨드를 이용한다.

- ```docker volume create [VOLUME_NAME]``` : volume을 생성한다.
- ```docker volume ls``` : 존재하는 volume의 리스트를 출력한다.
- ```docker volume inspect [VOLUME_NAME]``` : 해당 volume의 정보를 출력한다.
- ```-v [HOST_VOLUME_NAME]:[CONTAINER_MOUNT_POINT]:[MODE]``` : ```run``` command의 옵션으로 사용가능하고, container의 해당 지점에 볼륨을 마운트한다. 다른 container에서도 동시에 마운트할 수 있고, 데이터 공유관리가 가능하다. mode는 'ro'(read only), 'rw'(read and write)가 있다.
- ```docker volume rm [VOULME_NAME]``` : 해당 volume을 삭제한다.
- ```docker volume prune``` : 마운트되어있지 않은 모든 volume을 삭제한다.

두 번째 방법은 bind mount를 이용하는 방법이다. 이는 호스트의 파일시스템에서 특정 마운트 포인트를 지정하는 방식이다. 로컬에서는 이게 더 편리한 경우가 있다. 하지만 volume을 이용하면 docker가 해당 마운트를 관리해주기 때문에 대부분의 상황에서는 volume을 이용하는 것이 합리적이다.
- ```-v [HOST_MOUNT_POINT]:[CONTAINER_MOUNT_POINT]:[MODE]``` : ```run``` command의 옵션으로 사용가능하고, container의 해당 지점에 host의 서브 파일시스템을 마운트한다. mode는 'ro'(read only), 'rw'(read and write)가 있다.